# Exploration des données de Sirene

## Importation des différents packages nécessaires.

In [97]:
import pandas as pd
import dask.dataframe as dd
import random
import numpy as np
from datetime import datetime
from tqdm import tqdm
import nltk
import string
from nltk.corpus import stopwords
import fasttext
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/coder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Importation des données

In [2]:
DBRaw = dd.read_parquet('../data/extraction_sirene_20220510.parquet', engine='pyarrow')

On transforme les valeurs manquantes en NaN.

In [3]:
DBRaw = DBRaw.fillna(value=np.nan)

On garde seulement les variables potentiellement intéressantes : 
- ``APE_SICORE :`` Code APE (Activité Principale Exercée) retenu lors du traitement de codification (soit Sicore soit gestionnaire) ;
- ``NAT_SICORE :`` Nature de l'activité de l'entreprise ;
- ``SED_SICORE :`` Sédentarité de l'entreprise ;
- ``EVT_SICORE :`` Sédentarité de l'entreprise ;
- ``LIB_SICORE :`` Sédentarité de l'entreprise ;
- ``DATE :`` Sédentarité de l'entreprise ;
- ``AUTO :`` Type de liasse extrait de la base brute Sirène ;
- ``SURF :`` Surface en $m^2$ de l'établissement.


In [4]:
Var2Keep = ["APE_SICORE","LIB_SICORE","AUTO","DATE","NAT_SICORE","SED_SICORE","EVT_SICORE","SURF"]
DB = DBRaw[Var2Keep]

On supprime les liasses où une valeur est manquante pour l'une de ces deux variables (6.77% de la base). Il s'agit principalement du code APE donc il n'est pas nécessaire de l'imputer.

In [5]:
DB = DB.dropna(subset=['APE_SICORE'])  

On a finalement 10.8 millions de liasses.

In [6]:
DB.shape[0].compute()

10887847

## I- Modèle 1

On estime un modèle FastText standard en utilisant seulement les libellés comme features.

### 1) Preprocessing 

In [7]:
DB1 = DB[["APE_SICORE","LIB_SICORE"]]
DB1 = DB1.dropna(subset=['LIB_SICORE'])  

In [13]:
stopwords_ = set(stopwords.words('french') + ['a'])
def CleanLib(lib):
    # On supprime toutes les ponctuations
    lib = lib.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # On supprime tous les chiffres
    lib = lib.translate(str.maketrans(string.digits, ' ' * len(string.digits)))

    # On supprime les stopwords et on renvoie les mots en majuscule
    return " ".join([x.lower() for x in lib.split() if x.lower() not in stopwords_])

In [17]:
DB1["LIB_CLEAN"] = DB1["LIB_SICORE"].apply(lambda x: CleanLib(x), meta=pd.Series(dtype='str', name='LIB_CLEAN'))
df = DB1.compute()

In [18]:
df

,APE_SICORE,LIB_SICORE,LIB_CLEAN
LIA_NUM,,,
C00903254625,4799A,VENTE A DOMICILE ET/OU SUR MARCHES DE LINGERIE...,vente domicile marches lingerie sextoys maroqu...
U33071247848,7490B,CONSULTANT EN INNOVATION,consultant innovation
C00903254427,4782Z,"ACHAT ET VENTE DE TEXTILES, ACCESSOIRES ET DE ...",achat vente textiles accessoires tous produits...
C74019662643,7120B,LA CERTIFICATION DE BATIMENT GARANTISSANT LA P...,certification batiment garantissant performanc...
C91014088499,4619B,INTERMEDIAIRE DE COMMERCE MISE EN RELATION PRO...,intermediaire commerce mise relation producteu...
...,...,...,...
M69013888553,4299Z,AMENAGEMENT DE TERRAINS ET SALLE DE SPORT NOTA...,amenagement terrains salle sport notamment pos...
M25017641443,8211Z,SERVICES ADMINISTRATIFS DIVERS,services administratifs divers
M25017649123,9512Z,REPARATION DE TELEPHONES,reparation telephones


### 2) Splitting

On mélange de manière aléatoires les index puis on les divise en 3 groupes selon un certain pourcentage (ici 70%, 20% et 10%)


In [70]:
random.seed(123456)
Idx = random.sample(df.index.values.tolist(), df.shape[0])
Groups = np.split(Idx, [int(len(Idx)*0.7), int(len(Idx)*0.9)])

In [91]:
with open("../data/train_text.txt", 'w') as f:
    for idx in range(len(Groups[0])):
        aLine = "__label__{} {}".format(df.at[Groups[0][idx],"APE_SICORE"], df.at[Groups[0][idx],"LIB_CLEAN"])
        f.write("%s\n" % aLine)


In [92]:
with open("../data/test_text.txt", 'w') as f:
    for idx in range(len(Groups[1])):
        aLine = "__label__{} {}".format(df.at[Groups[1][idx],"APE_SICORE"], df.at[Groups[1][idx],"LIB_CLEAN"])
        f.write("%s\n" % aLine)

In [93]:
with open("../data/validation_text.txt", 'w') as f:
    for idx in range(len(Groups[2])):
        aLine = "__label__{} {}".format(df.at[Groups[2][idx],"APE_SICORE"], df.at[Groups[2][idx],"LIB_CLEAN"])
        f.write("%s\n" % aLine)

### 3) Training

On définit plusieurs options pour le modèle.

In [94]:
config_fasttext={"dim": 150,
"lr": 0.2,
"epoch": 80,
"wordNgrams": 3,
"minn": 3,
"maxn": 4,
"minCount": 3,
"bucket": 3000000,
"thread": 25,
"loss": 'ova',
"label_prefix": '__label__'}

In [98]:
model1 = fasttext.train_supervised(input="../data/train_text.txt", **config_fasttext)

Read 54M words
Number of words:  52388
Number of labels: 727
Progress:   4.4% words/sec/thread:   11253 lr:  0.191228 avg.loss:  2.714532 ETA:   4h 8m30s avg.loss:  2.345083 ETA:   4h19m21ss 0.192981 avg.loss:  2.554645 ETA:   4h10m44s 8m50s